<h1 style="color:red;">13th and 14th March</h1>

In [1]:
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from datetime import datetime
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import gc
import glob
import imageio.v2
import os
from IPython import display
import netCDF4

### *Path to the file*

In [2]:
%%time
path = "/glade/work/noteng/research/data/"
### name of the file ###
file = 'march13-march14.nc'

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.48 µs


### *Open netcdf data*

In [3]:
%%time
data = xr.open_dataset(path+file)

CPU times: user 21.5 ms, sys: 20.6 ms, total: 42.1 ms
Wall time: 172 ms


<!-- # saved the file to my local system -->

### *Equivalent Reflectivity Factor from data variables*

In [4]:
%%time
equi_ref_factor = data['equivalent_reflectivity_factor'].load()
equi_ref_factor

CPU times: user 24.3 s, sys: 3.37 s, total: 27.7 s
Wall time: 27.8 s


<xarray.DataArray 'equivalent_reflectivity_factor' (time: 360, Yc: 2243,
                                                    Xc: 2067)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13 ... 2020-03-14T05:55:00
  * Xc       (Xc) float32 -5.33e+05 -5.32e+05 -5.31e+05 ... 1.532e+06 1.533e+06
  * Yc       (Yc) float32 -1.81e+06 -1.811e+06 ... -4.051e+06 -4.052e+06
    lon      (Yc, Xc) float32 -6.408 -6.379 -6.35 -6.321 ... 30.7 30.71 30.72
    lat      (Yc, Xc) float32 73.04 73.04 73.05 73.05 ... 50.38 50.38 50.37
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

## *Figure*

In [9]:
%%time
# for i in range(0, 1):
for i in range(0, len(equi_ref_factor)):
    ### center points and extent of map ###
    latA = 69.141281  # latitude of COMBLE site
    lonA = 15.684166-1  # longitude of COMBLE site -1
    xm, ym = 6, 2.75  # extent from center point in lon and lat

    fig, ax = plt.subplots(1, 1, figsize=(10, 9), subplot_kw={
                           'projection': ccrs.Orthographic(lonA, latA)})

# for i in range(277, 287):
    extent = (lonA-xm, lonA+xm, latA-ym, latA+ym)  # define extent map
    ax.set_extent(extent)  # set extent of map
    # plot coastlines with high resolution: 10m
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS)
    
    ax.add_feature(cfeature.LAND, linewidth=1.5, alpha=0.5)
  
    # add grid lines
    gl = ax.gridlines(x_inline=False, alpha=0.3); gl.bottom_labels=True; gl.left_labels=True
    gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'rotation': 45}; gl.ylabel_style = {'color': 'black'}
    

#     # change time to moth, day, year, hours and minutes
    dt = equi_ref_factor['time'].dt.strftime('%m-%d-%Y %H:%M').values  # array

    cs = ax.pcolormesh(equi_ref_factor['lon'],
                       equi_ref_factor['lat'],
                       equi_ref_factor[i],
                       cmap='jet',
                       transform=ccrs.PlateCarree(),
                       vmin=-20,
                       vmax=40
                       )

#     ax.plot(lonA+1, latA, color='red', marker='*', markersize=10.5,
#             transform=ccrs.PlateCarree())  # plot red star at location of Andenes
    lat1 = 69.24139; lon1 = 16.00297
    # ax.plot(lonA+1, latA, color='red', marker='*', markersize=10.5,
    #         transform=ccrs.PlateCarree())  # plot red star at location of Andenes
    ax.plot(lon1, lat1, color='red', marker='*', markersize=10.5,
            transform=ccrs.PlateCarree())  # plot red star at location of Radar Andoya
    # 59.7264 N, 10.4544 E
    ax.plot(12.1544, 67.5264, color='red', marker='*', markersize=10.5,
            transform=ccrs.PlateCarree())  # plot red star at location at SW
    
   
    ax.plot(21.7544, 70.664, color='red', marker='*', markersize=10.5,
            transform=ccrs.PlateCarree())  # plot red star at location at NE
    
#     ax.plot(20.7544, 67.7264, color='red', marker='*', markersize=10.5,
#             transform=ccrs.PlateCarree())  # plot red star at location at SE
    
#     # title of figure
    ax.set_title(
        f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)

#     # Add a colorbar axis at the right of the graph
    cbar_ax = fig.add_axes([0.85, 0.12, 0.047, 0.75])  # lbwh

    cbar = fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
    cbar.set_label("Reflectivity Factor [dBZ]", fontsize=15)

    dirr = sorted(os.listdir('./img-mar13-mar14///'))
    if f'{dt[i]}.png' in dirr:
        # print(f'{dt[i]}.png already exist')
        pass
        plt.close()
    else:
        fig.savefig(f'./img-mar13-mar14///{dt[i]}.png', bbox_inches='tight',dpi=300)
        plt.close()
print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
gc.collect()

😀😀😀😀EXECUTION DONE!!!😀😀😀😀
CPU times: user 48min 3s, sys: 2min 36s, total: 50min 39s
Wall time: 50min 40s


15

<h2 style="color:red;font-weight:bold;"><em>Display Animation</em></h2>

In [10]:
%%time
# Build GIF
imgs = sorted(glob.glob("./img-mar13-mar14/*.png"))
with imageio.get_writer('march-13th-14th-animation.gif', mode='I', duration=0.3) as writer:
    for filename in imgs:
        image = imageio.v2.imread(filename)
        writer.append_data(image)

CPU times: user 4min 56s, sys: 245 ms, total: 4min 56s
Wall time: 5min 4s


In [ ]:
%%time
#Display image
display.Image('march-13th-14th-animation.gif', width=700)

CPU times: user 3.12 ms, sys: 96.1 ms, total: 99.2 ms
Wall time: 131 ms
